In [1]:
import pickle
import pandas as pd
from sklearn.metrics import confusion_matrix
import copy
import sys
sys.path.append('../')
from scripts.tools import *
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt  
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
def predict(X):

    #predict which class 
    class_predictions = class_pred_rf.predict(X)
    
    #predict if class is a invalid class based on prob distributions
    prob_predicitions = class_pred_rf.predict_proba(X)

    #prepare inputs for invalid_rf 
    #inputs [class probabilities, number of zeros, max prob]
    _input = []
    for prob in prob_predicitions:
        tmp = []
        prob = list(prob)
        tmp.extend(prob)
        tmp.append(prob.count(0))
        tmp.append(max(prob))
        _input.append(tmp)


    invalid_predictions = invalid_rf.predict(_input)

    pred_classes = []
    for inv,_class in zip(invalid_predictions,class_predictions):
        if inv == 'invalid':
            pred_classes.append('invalid')
        else:
            pred_classes.append(_class)
    
    return pred_classes

In [3]:
clf = pickle.load(open('../models/class_pred_rf.sav', 'rb'))
class_pred_rf = copy.deepcopy(clf)
invalid_rf = pickle.load(open('../models/invalid_pred_rf.sav', 'rb'))

In [4]:
clf.predict = predict

In [ ]:
INPUT_PATH = '../data/train.csv'
df = pd.read_csv(INPUT_PATH)

Error: Session cannot generate requests

Accuracy analysis on validatio test set of (train.csv)
------------------------

In [5]:
X = df.values
X = [x[:-1] for x in X] # remove cuisine
Y = list(df['cuisine'])
Y = [y.lower() for y in Y]

In [ ]:
# show_class_ratio(Y)

Error: Session cannot generate requests

In [ ]:

# X,Y = cap_dataset(X,Y,500)

In [ ]:
_, X_test, _, Y_test = train_test_split(X, Y, test_size=0.05, random_state=0)


Error: Session cannot generate requests

In [ ]:
scores = cross_val_score(clf, X, Y, cv=5,n_jobs=-2)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
Y_pred = clf.predict(X)
accuracy = accuracy_score(Y, Y_pred, normalize=True)
print("Accuracy:",accuracy)

Error: Session cannot generate requests

Accuracy analysis on validation.csv
------------------------

In [ ]:
INPUT_PATH = '../data/cleaned_validation.csv'

df = pd.read_csv(INPUT_PATH)

print(len(df.columns))

X = df.values
print(f'n samples: {len(X)}')
X = [x[:-1] for x in X] # remove cuisine
Y = list(df['cuisine'])
Y = [y.lower() for y in Y]


FileNotFoundError: [Errno 2] No such file or directory: '../data/cleaned_validation.csv'

In [ ]:
X,Y = cap_dataset(X,Y,500)

In [ ]:
scores = cross_val_score(clf, X, Y, cv=5,n_jobs=27)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.60 accuracy with a standard deviation of 0.01


In [ ]:
Y_pred = clf.predict(X)
accuracy = accuracy_score(Y, Y_pred, normalize=True)

print("\n#########fited#########\n")
print("Accuracy:",accuracy)


#########fited#########

Accuracy: 0.7638230647709321


In [ ]:
def show_wrongly_predicted(X, Y, model):

    predictions = model.predict(X)

    wrong_pred = {}
    for pred, y in zip(predictions, Y):
        if pred != y:
            wrong_pred.setdefault(y, 0)
            wrong_pred[y] += 1

    train_ratios = class_ratio(Y)

    cr = {r[0]: int(r[1]) for r in train_ratios}

    print(' ')
    # n times wrongly classified
    for y in wrong_pred:
        wa = wrong_pred[y]
        print(
            f'* wrongly predicted {y}: {wa}/{cr[y]}:  {int(round(wa/cr[y],3)*100)}%')


In [ ]:
show_wrongly_predicted(X,Y,clf)

 
* wrongly predicted indian: 70/500:  14%
* wrongly predicted chinese: 52/139:  37%
* wrongly predicted invalid: 16/500:  3%
* wrongly predicted french: 109/135:  80%
* wrongly predicted mexican: 62/427:  14%
* wrongly predicted italian: 112/421:  26%
* wrongly predicted japanese: 18/37:  48%
* wrongly predicted irish: 19/32:  59%
* wrongly predicted thai: 43/108:  39%
* wrongly predicted moroccan: 3/7:  42%
* wrongly predicted british: 5/5:  100%
* wrongly predicted brazilian: 4/6:  66%
* wrongly predicted greek: 22/53:  41%
* wrongly predicted filipino: 21/78:  26%
* wrongly predicted spanish: 19/42:  45%
* wrongly predicted vietnamese: 4/11:  36%
* wrongly predicted korean: 5/9:  55%
* wrongly predicted russian: 14/21:  66%


In [ ]:
def class_or_not(y):
    if y != 'invalid':
        return True
    else: 
        return False 

Recall, Precision of class vs invalid
----------

In [ ]:
#Precision on class vs invalid

Y_pred = clf.predict(X)

TP = 0
FP = 0

for gt,pred in zip(Y,Y_pred):
    # if data point is a real class
    if class_or_not(gt) and class_or_not(pred):
        TP += 1
    elif not class_or_not(gt) and class_or_not(pred):
        FP += 1
print(TP/(TP+FP))


0.992


99% of predicted classes are valid classes

In [ ]:
#Recall on class vs invalid

Y_pred = clf.predict(X)

TP = 0
FN = 0

for gt,pred in zip(Y,Y_pred):
    # if data point is a real class
    if class_or_not(gt) and class_or_not(pred):
        TP += 1
    elif class_or_not(gt) and not class_or_not(pred):
        FN += 1
print(TP/(TP+FN))

0.9763779527559056


97% of all classes are correcty identified


Recall, precision of all classes (including invalid)
------

In [ ]:
labels = list(set(Y))
labels.append('invalid')
print(labels)
rs = recall_score(Y, Y_pred, average='macro',labels=labels)
print(rs)

['mexican', 'italian', 'japanese', 'greek', 'spanish', 'chinese', 'russian', 'filipino', 'british', 'vietnamese', 'thai', 'moroccan', 'invalid', 'brazilian', 'irish', 'french', 'indian', 'korean', 'invalid']
0.5740742857348194


In [ ]:

ps = precision_score(Y, Y_pred, average='macro',labels=labels)
print(ps)

0.6606323182307537


In [ ]:
fs = f1_score(Y, Y_pred, average='macro',labels=labels)
print(fs)

0.5983293076474165
